In [1]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=56d773936f804e605f4d5f09f967633cc7cb379458ddacacda6396a438553177
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [7]:
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

### Create the schema of the streamed files (check the column names and types from the CSV files)

In [3]:
!unzip kospi.zip

Archive:  kospi.zip
   creating: kospi/
  inflating: kospi/KOSPI_STOCK_8.csv  
  inflating: kospi/KOSPI_STOCK_10.csv  
  inflating: kospi/KOSPI_STOCK_5.csv  
  inflating: kospi/KOSPI_STOCK_7.csv  
  inflating: kospi/KOSPI_STOCK_9.csv  
  inflating: kospi/KOSPI_STOCK_2.csv  
  inflating: kospi/KOSPI_STOCK_3.csv  
  inflating: kospi/KOSPI_STOCK_6.csv  
  inflating: kospi/KOSPI_STOCK_4.csv  
  inflating: kospi/KOSPI_STOCK_1.csv  
  inflating: kospi/KOSPI_STOCK_0.csv  


In [31]:
mySchema = "ID int, Date date, Open double, High double, Low double, Close double, Adj_Close double, Volume int"

### Create the dataframe by reading the stream using format "csv" and the schema you created.

In [9]:
stream_df = spark.readStream \
    .format("csv") \
    .schema(mySchema)\
    .option("header", "true") \
    .load('/content/kospi')

### Make sure the dataframe is streaming the files from the folder

In [10]:
stream_df.isStreaming

True

### Create a stream writer into memory and specify the query name "stock:

In [11]:
writer = stream_df.writeStream.format('memory')\
.queryName('stock')\
.option('numRows', 200)\
.trigger(processingTime='300 seconds')\
.outputMode('append')

### Start the write stream and make sure it works (read all columns from the table)

In [17]:
q = writer.start()

In [18]:
spark.sql("SELECT * FROM stock").show(100, truncate=False)

+---+----------+------------+------------+------------+------------+------------+------+
|ID |Date      |Open        |High        |Low         |Close       |Adj_Close   |Volume|
+---+----------+------------+------------+------------+------------+------------+------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797|91019 |
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328|40656 |
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|
|244|2000-12-11|27687.099609|28860.0     |27651.599609|28078.099609|26533.740234|159671|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328|74560 |
|246|2000-12-13|27651.599609|29286.5     |27651.599609|28469.099609|26903.234375|270385|
|247|2000-12-14|28469.099609|29784.099609|28291.300781|28362.400391|26802.40625 |256317|
|248|2000-12-15|28362

In [19]:
q.stop()

### Remove the first row from the data (hint: drop the rows where ALL values are null), then add a new column "diff", which is the difference between high and low columns

In [21]:
dropfirstrow_df = stream_df.dropna(how='all')
diff_df = dropfirstrow_df.withColumn("diff", col("High") - col("Low"))

### Create a new write stream using the new generated dataframe and call the generate table "modified_data"

In [23]:
second_writer = diff_df.writeStream.format('memory')\
.queryName('modified_data')\
.option('numRows', 200)\
.trigger(processingTime='30 seconds')\
.outputMode('append')

In [35]:
new_query = second_writer.start()

In [36]:
spark.sql("SELECT * FROM modified_data").show(100, truncate=False)

+---+----------+------------+------------+------------+------------+------------+------+------------------+
|ID |Date      |Open        |High        |Low         |Close       |Adj_Close   |Volume|diff              |
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797|91019 |995.201172000001  |
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791|710.8007819999984 |
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328|40656 |533.1015620000035 |
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964|1066.298827999999 |
|244|2000-12-11|27687.099609|28860.0     |27651.599609|28078.099609|26533.740234|159671|1208.4003909999992|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328|74560 |639.798827999999  |
|246|2000-12-13|27651.599609

In [26]:
new_query.stop()

### Write the generated data into files instead of the memory.

In [37]:
third_writer = diff_df.writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", "/content/output/kospi_data") \
    .option("CheckpointLocation", "/content/checkpoints/kospi_data")

In [42]:
third_query = third_writer.start()

In [39]:
third_query.stop()

### Stop the query. Now, try reading the generated files into a normal dataframe
- Create a schema and use it to read the data.
- Show the output.

In [48]:
mySchema = "ID int, Date date, Open double, High double, Low double, Close double, Adj_Close double, Volume int, diff double"

In [49]:
written_df= spark.read\
    .schema(mySchema)\
    .csv('/content/output/kospi_data')

In [50]:
written_df.isStreaming

False

In [51]:
written_df.show(25)

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj_Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|240|2000-12-05|26585.300781|27367.300781|26372.099609|27011.800781|25526.091797| 91019|  995.201172000001|
|241|2000-12-06|27011.800781|27509.400391|26798.599609|26869.699219|25391.804688|105791| 710.8007819999984|
|242|2000-12-07|27011.800781|27011.800781|26478.699219|26656.400391|25190.236328| 40656| 533.1015620000035|
|243|2000-12-08|26656.400391|27722.699219|26656.400391|27651.599609|26130.699219|149964| 1066.298827999999|
|244|2000-12-11|27687.099609|     28860.0|27651.599609|28078.099609|26533.740234|159671|1208.4003909999992|
|245|2000-12-12|28042.599609|28078.099609|27438.300781|27935.900391|26399.361328| 74560|  639.798827999999|
|246|2000-12-13|27651.599609

### Sort the dataframe based on the ID

In [52]:
finalDFSorted = written_df.sort('ID')
finalDFSorted.show()

+---+----------+------------+------------+------------+------------+------------+------+------------------+
| ID|      Date|        Open|        High|         Low|       Close|   Adj_Close|Volume|              diff|
+---+----------+------------+------------+------------+------------+------------+------+------------------+
|  0|2000-01-04|22817.900391|25696.800781|22817.900391|24879.300781|23510.880859|108745|2878.9003900000025|
|  1|2000-01-05|24523.900391|26229.900391|23670.900391|24417.300781|23074.294922|175990|            2559.0|
|  2|2000-01-06|24381.699219|24666.099609|22746.800781|22817.900391|21562.865234| 71746| 1919.298827999999|
|  3|2000-01-07|     22036.0|24879.300781|     22036.0|23884.199219|22570.513672|120984|2843.3007810000017|
|  4|2000-01-10|24879.300781|25519.099609|23813.099609|24061.900391|22738.439453|151371|            1706.0|
|  5|2000-01-11|     24168.5|     25021.5|23955.199219|24239.599609|22906.365234| 95943|1066.3007810000017|
|  6|2000-01-12|     24168.5